<a href="https://colab.research.google.com/github/Yewon9/Emotion_Recognition/blob/main/Text_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install sentencepiece

In [3]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
import re
import pandas as pd

# DATA

In [5]:
df = pd.read_csv('emotion_text.csv')
df

,Emotion,Path,Length,Text
0,angry,/content/drive/MyDrive/project/data_aihub/4/5e...,4.565333,10년도 넘었어 고등학교 동창
1,angry,/content/drive/MyDrive/project/data_aihub/4/5e...,8.576000,10분마다 연락을 해 봤지 근데 아직도이 녀석 온다고 말하지 오지 않고 있어
2,angry,/content/drive/MyDrive/project/data_aihub/4/5e...,7.040000,1년 채우고 그만두려고 했는데 아 지금 같아서는 진짜 회사 옮기고 싶다
3,angry,/content/drive/MyDrive/project/data_aihub/4/5e...,5.034667,1시간 기다렸나 봐 다리도 아프고 짜증 나
4,angry,/content/drive/MyDrive/project/data_aihub/4/5e...,4.266667,1시간 기다렸는데 아직도 안 와
...,...,...,...,...
14377,sadness,/content/drive/MyDrive/project/data_aihub/4/5e...,10.880000,힘들다 그만둬야 할까 어떻게 해야 될까 정말 고민이 많이 되는 거 같아 어떡하면 좋겠니
14378,sadness,/content/drive/MyDrive/project/data_aihub/5_2/...,4.821333,힘들다 힘들어
14379,sadness,/content/drive/MyDrive/project/data_aihub/4/5e...,5.248000,힘들어 다른 회사 또 준비를 할 거야 나는
14380,sadness,/content/drive/MyDrive/project/data_aihub/4/5e...,3.114667,힘들어하고 나도 너무 힘들어


In [6]:
empty_or_error_rows = df[(df['Text'].str.strip() == '') | (df['Text'] == 'Error')]
print(len(empty_or_error_rows))

0


In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\s]', '', text)
    return text

In [9]:
df['cleaned_text'] = df['Text'].apply(clean_text)

In [11]:
print(df[['Text', 'cleaned_text']].head())

                                         Text  \
0                            10년도 넘었어 고등학교 동창   
1  10분마다 연락을 해 봤지 근데 아직도이 녀석 온다고 말하지 오지 않고 있어   
2     1년 채우고 그만두려고 했는데 아 지금 같아서는 진짜 회사 옮기고 싶다   
3                     1시간 기다렸나 봐 다리도 아프고 짜증 나   
4                           1시간 기다렸는데 아직도 안 와   

                                 cleaned_text  
0                            10년도 넘었어 고등학교 동창  
1  10분마다 연락을 해 봤지 근데 아직도이 녀석 온다고 말하지 오지 않고 있어  
2     1년 채우고 그만두려고 했는데 아 지금 같아서는 진짜 회사 옮기고 싶다  
3                     1시간 기다렸나 봐 다리도 아프고 짜증 나  
4                           1시간 기다렸는데 아직도 안 와  


In [12]:
def tokenize_text(text):
    return tokenizer(text, padding = 'max_length', truncation = True, max_length = 128, return_tensors = "pt")

In [14]:
df['tokenized'] = df['cleaned_text'].apply(tokenize_text)

In [15]:
print(df['tokenized'].head())

0    [input_ids, token_type_ids, attention_mask]
1    [input_ids, token_type_ids, attention_mask]
2    [input_ids, token_type_ids, attention_mask]
3    [input_ids, token_type_ids, attention_mask]
4    [input_ids, token_type_ids, attention_mask]
Name: tokenized, dtype: object


In [17]:
df.to_csv('emotion_tokenized_data.csv', index = False, encoding = 'utf-8-sig')